# Практическое задание 3 

# Классификация предложений с использованием BERT

## курс "Математические методы анализа текстов"


### ФИО: Хуршудов Артем Эрнестович

## Введение

### Постановка задачи

В этом задании вы будете классифицировать предложения из медицинских статей на несколько классов (background, objective и т.д.). 
Для того, чтобы улучшить качество решения вам предлагается дообучить предобученную нейросетевую архитектуру BERT.

### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
 - [Pytorch](https://pytorch.org/).
 - [Transformers](https://github.com/huggingface/transformers).
 
### Данные

Скачать данные можно здесь: [ссылка на google диск](https://drive.google.com/file/d/13HlWH8jnmsxqDKrEptxOXQg9kkuQMmGq/view?usp=sharing)

## Часть 1. Подготовка данных

Мы будем работать с предложениями из медицинских статей, разбитых на несколько классов. 

In [0]:
import re
from collections import Counter

Путь к папке с данными:

In [0]:
DATA_PATH = "./"

Функция считывания данных:

In [0]:
def read_data(file_name):
    """
    Parameters
    ----------
    file_name : str
        Pubmed sentences file path
        
    Returns
    -------
    text_data : list of str
        List of sentences for algorithm
    
    target_data : list of str
        List of sentence categories
    """
    text_data = []
    target_data = []

    with open(file_name, 'r') as f_input:
        for line in f_input:
            if line.startswith('#') or line == '\n':
                continue
            target, text = line.split('\t')[:2]    

            text_data.append(text)
            target_data.append(target)
    
    return text_data, target_data

Считывание данных:

In [0]:
train_data, train_target = read_data(f'{DATA_PATH}/data_train.txt')
test_data, test_target = read_data(f'{DATA_PATH}/data_test.txt')
dev_data, dev_target = read_data(f'{DATA_PATH}/data_dev.txt')

## Часть 2. Построение бейзлайна (1 балл)

В этой части задания вам необходимо построить бейзлайн модель, с которой вы будете сравнивать ваше решение. В качестве бейзлайна вам предлагается использовать модель логистической регрессии на tf-idf представлениях.

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

Перед тем как подать в модель предложения, необходимо их предобработать:
    
1. привести все предложения к нижнему регистру
2. удалить из предложений все непробельные символы кроме букв, цифр
3. все цифры заменить на нули

Метки ответов необходимо преобразовать из текстового вида в числовой (это можно сделать с помощью LabelEncoder).

Затем необходимо построить tf-idf матрицу по выбранным предложениям (используйте для подсчёта tf-idf только train_data!) и обучить на них модель логистической регрессии. Используйте dev выборку для подбора гиперпараметров модели. Добейтесь того, что на test и dev выборках accuracy будет будет выше 0.8.

In [0]:
le = LabelEncoder()
train_target = le.fit_transform(train_target)
test_target = le.transform(test_target)
dev_target = le.transform(dev_target)

In [0]:
import re

regex1 = re.compile('[^a-z0-9\s]')
regex2 = re.compile('[0-9]')

def preprocess(text):
    text = text.lower()
    text = regex1.sub('', text)
    text = regex2.sub('0', text)
    
    return text

In [0]:
train_data_prep = [preprocess(el) for el in train_data]
test_data_prep = [preprocess(el) for el in test_data]
dev_data_prep = [preprocess(el) for el in dev_data]

# train_data = [preprocess(el) for el in train_data]
# test_data = [preprocess(el) for el in test_data]
# dev_data = [preprocess(el) for el in dev_data]

In [0]:
tf = TfidfVectorizer(ngram_range=(1,2))
train_data_tf = tf.fit_transform(train_data_prep)
test_data_tf = tf.transform(test_data_prep)
dev_data_tf = tf.transform(dev_data_prep)

In [0]:
clf = LogisticRegression(multi_class='auto', C=3)
clf.fit(train_data_tf, train_target)

print(accuracy_score(clf.predict(train_data_tf), train_target))
print(accuracy_score(clf.predict(dev_data_tf), dev_target))
print(accuracy_score(clf.predict(test_data_tf), test_target))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9568711219611433
0.8058205447255634
0.8034158978747348


## Часть 3. Задание BERT (4 балла за 3 и 4 части)

Так как обучающих предложений очень мало, попробуем использовать модель BERT, предобученную на большом датасете. Будем использовать библиотеку transformers. Для обучения модели используйте данные до обработки из предыдущего пункта.

In [0]:
# !pip install transformers

In [0]:
BERT_MODEL_NAME = "bert-base-uncased"
NUM_LABELS = len(set(train_target))

In [0]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, WarmupLinearSchedule

import torch
from torch.utils.data import DataLoader, Dataset

Модель BERT работает с специальным форматом данных — все токены из предложения получены с помощью алгоритма BPE. Класс BertTokenizer позволяет получить BPE разбиение для предложения.

In [0]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

100%|██████████| 231508/231508 [00:00<00:00, 426041.83B/s]


В библиотеке transformers есть специальный класс для работы с задачей классификации — BertForSequenceClassification. Воспользуемся им, чтобы задать модель.

In [0]:
bert_model = BertForSequenceClassification.from_pretrained(
    BERT_MODEL_NAME, num_labels=NUM_LABELS
)

bert_model.to('cuda')
##

100%|██████████| 440473133/440473133 [00:36<00:00, 12073676.08B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Реализуем специальный кастомный датасет для токенизированных с помощью BPE предложений. Каждое предложение должно быть преобразовано в последовательность BPE индексов. Не забудьте, что в начале каждого предложения должен стоять специальный токен [CLS], а в конце должен стоять специальный токен [SEP].

Задайте датасет, используя BertTokenizer:

In [0]:
class BertTokenizedDataset(Dataset):
    def __init__(self, tokenizer, text_data, target_data=None, max_length=256):
        """
        Parameters
        ----------
        tokenizer : instance of BertTokenizer
        text_data : list of str
            List of input sentences
        target_data : list of int
            List of input targets
        max_length : int
            Maximum length of input sequence (length in bpe tokens)
        """

        self.data = []
        self.target_data = target_data

        res_arr = []
        for el in text_data:
          self.data.append(torch.tensor(tokenizer.encode(el, max_length = max_length, add_special_tokens=True)))
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        if self.target_data is not None:
            return self.data[i], self.target_data[i]
        else:
            return self.data[i]

Получите все датасеты для всех типов данных. 

**Замечание**. После получения есть смысл сохранить все датасеты на диск, т.к. предобработка занимает время.

In [0]:
train_dataset = BertTokenizedDataset(tokenizer, train_data, train_target)
dev_dataset = BertTokenizedDataset(tokenizer, dev_data, dev_target)
test_dataset = BertTokenizedDataset(tokenizer, test_data, test_target)

Используем  класс PadSequences, чтобы задать способ паддинга, работающий с встроенным в pytorch DataLoader.

In [0]:
class PadSequences:
    def __init__(self, use_labels=False):
        self.use_labels = use_labels
    
    def __call__(self, batch):
        """
        Parameters
        ----------
        batch : list of objects or list of (object, label)
            Each object is list of int indexes.
            Each label is int.
        """
        data_label_batch = batch if self.use_labels else [(x, 0) for x in batch]

            
        # Sort the batch in the descending order
        sorted_batch = sorted(data_label_batch, key=lambda x: x[0].shape[0], reverse=True)
        # Get each sequence and pad it
        sequences = [x[0] for x in sorted_batch]
        sequences_padded = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True)
        max_lenght = len(sequences[0])

        # Also need to store the length of each sequence
        # This is later needed in order to unpad the sequences
        lengths = torch.LongTensor([[1] * len(x) + [0] * (max_lenght - len(x)) for x in sequences])
        # Don't forget to grab the labels of the *sorted* batch
        
        if self.use_labels:
            labels = torch.LongTensor([x[1] for x in sorted_batch])
            return sequences_padded, lengths, labels
        else:
            return sequences_padded

Зададим DataLoader для каждого из датасетов:

In [0]:
BATCH_SIZE = 16

In [0]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                              collate_fn=PadSequences(use_labels=True))

dev_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=PadSequences(use_labels=True))

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=PadSequences(use_labels=True))

Заметьте, что модель трансформера обучается по достаточному большому размеру батча (обычно 64), который скорее всего не будет влезать на вашу видеокарту. Поэтому, рекомендуется "накапливать" градиенты за несколько итераций. С помощью параметра ACCUMULATION_STEPS задайте, раз в сколько итераций вам необходимо делать шаг метода оптимизации.

In [0]:
EPOCH_AMOUNT = 1
TRAIN_LENGTH = len(train_dataset)
ACCUMULATION_STEPS = 4

LR = 2e-5

Посчитайте общее число раз, когда ваш оптимизатор будет делать обновления на основе выбранных значений EPOCH_AMOUNT, BATCH_SIZE, ACCUMULATION_STEPS и  TRAIN_LENGTH. Эта величина нужна для правильного задания параметров оптимизаторов.

In [0]:
train_optimization_step_amount = TRAIN_LENGTH / BATCH_SIZE / ACCUMULATION_STEPS * EPOCH_AMOUNT

Зададим параметры оптимизаторов. Мы будем использовать специальные оптимизаторы из библиотеки transformers AdamW и WarmupLinearSchedule, обеспечивающие плавный разгон и медленное затухание темпа обучения.

Для некоторых групп параметров зададим коэффициенты регуляризации.

In [0]:
param_optimizer = list(bert_model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(bert_model.parameters(),
                     lr=LR)
scheduler = WarmupLinearSchedule(
    optimizer,
    warmup_steps=train_optimization_step_amount * 0.05,
    t_total=train_optimization_step_amount,
)

## Часть 4. Обучение BERT 

Теперь всё готово к тому, чтобы дообучить BERT на датасете train_dataset!

Используйте dev_dataset для выбора гиперпараметров модели и обучения. Задание будет засчтано на полный балл если на dev_dataset и test_dataset точность будет выше 0.84.

In [0]:
%%time

device = 'cuda'
losses = []

for epoch in range(EPOCH_AMOUNT):  
  
  bert_model.train()  
  for batch in train_dataloader:

    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    bert_model.train()
    outputs = bert_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    
    loss, logits = outputs[:2]

    losses.append(loss)

    loss.backward()
    # torch.nn.utils.clip_grad_norm_(bert_model.parameters(), max_grad_norm)  # Gradient clipping is not in AdamW anymore (so you can use amp without issue)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

In [0]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [0]:
## VALIDATION
import numpy as np
from sklearn.metrics import accuracy_score

bert_model.eval()

pred = []
real = []

for batch in dev_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad():
    logits = bert_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)[0]

  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  pred_flat = np.argmax(logits, axis=1)
  pred.append(pred_flat)
  real.append(label_ids)

print(accuracy_score(flatten(pred), flatten(real) ))

0.8546246370800498


In [0]:
## test
import numpy as np
from sklearn.metrics import accuracy_score

bert_model.eval()

pred = []
real = []

qqq = len(test_dataloader) / 10

for it, batch in enumerate(test_dataloader):
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad():
    logits = bert_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)[0]

  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  pred_flat = np.argmax(logits, axis=1)
  pred.append(pred_flat)
  real.append(label_ids)

print(accuracy_score(flatten(pred), flatten(real)))

0.8509325143305942


## Бонусная часть (до 3 баллов)

Улучшите качество (на обеих выборках), используя любые способы (кроме использования дополнительных обучающих данных датасета RCT2000):

* $> 0.86$ — 1 балл 
* $> 0.88$ — 2 балла
* $> 0.9$ — 3 балла

In [0]:
BATCH_SIZE = 16

train_dataset = BertTokenizedDataset(tokenizer, train_data, train_target)
dev_dataset = BertTokenizedDataset(tokenizer, dev_data, dev_target)
test_dataset = BertTokenizedDataset(tokenizer, test_data, test_target)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                              collate_fn=PadSequences(use_labels=True))

dev_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=PadSequences(use_labels=True))

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=PadSequences(use_labels=True))

EPOCH_AMOUNT = 2
TRAIN_LENGTH = len(train_dataset)
ACCUMULATION_STEPS = 1

LR = 2e-5

train_optimization_step_amount = TRAIN_LENGTH / BATCH_SIZE / ACCUMULATION_STEPS * EPOCH_AMOUNT

In [0]:
bert_model = BertForSequenceClassification.from_pretrained(
    BERT_MODEL_NAME, num_labels=NUM_LABELS
)

bert_model.to('cuda')
print('ok')

In [0]:
param_optimizer = list(bert_model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(#bert_model.parameters(),
                  optimizer_grouped_parameters,
                  lr=LR, correct_bias=False)
scheduler = WarmupLinearSchedule(
    optimizer,
    warmup_steps=train_optimization_step_amount * 0.05,
    t_total=train_optimization_step_amount,
)

In [0]:
import numpy as np

def count_acc(model, data_loader, it_max=10000000):
  flatten = lambda l: [item for sublist in l for item in sublist]

  model.eval()

  pred = np.array([])
  real = np.array([])

  for it, batch in enumerate(data_loader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)[0]

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    pred_flat = np.argmax(logits, axis=1)

    pred = np.append(pred, [pred_flat])
    real = np.append(real, [label_ids])

    if it > it_max:
      break

  return accuracy_score(pred.reshape(-1), real.reshape(-1))

In [24]:
# %%time
loss_arr_1 = []

device = 'cuda'

for epoch in range(EPOCH_AMOUNT):  
  
  bert_model.train()  
  for idx, batch in enumerate(train_dataloader):

    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    bert_model.train()
    outputs = bert_model(b_input_ids, token_type_ids=None, labels=b_labels)
    
    loss, logits = outputs[:2]

    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

    loss_arr_1.append(loss)

    if idx % 100 == 0:
      print(epoch, idx)
      c1 = count_acc(bert_model, dev_dataloader, it_max=30)
      c2 = count_acc(bert_model, test_dataloader, it_max=30)
      print('c1:', c1)
      print('c2:', c2)
      print(loss)
      # if epoch < 1:
      #   continue
      # if (c1 > 0.86) and (c2 > 0.86):
      #   c3 = count_acc(bert_model, dev_dataloader)
      #   print('c3:',c3)
      #   if c3 > 0.86:
      #     c4 = count_acc(bert_model, test_dataloader)
      #     print('c4:',c4)
      #     if c4 > 0.86:
      #       break
      print('-' * 100)

0 0
c1: 0.107421875
c2: 0.138671875
tensor(1.6213, device='cuda:0', grad_fn=<NllLossBackward>)
----------------------------------------------------------------------------------------------------
0 100
c1: 0.775390625
c2: 0.767578125
tensor(0.6149, device='cuda:0', grad_fn=<NllLossBackward>)
----------------------------------------------------------------------------------------------------
0 200
c1: 0.8046875
c2: 0.767578125
tensor(0.6834, device='cuda:0', grad_fn=<NllLossBackward>)
----------------------------------------------------------------------------------------------------
0 300
c1: 0.818359375
c2: 0.841796875
tensor(0.7707, device='cuda:0', grad_fn=<NllLossBackward>)
----------------------------------------------------------------------------------------------------
0 400
c1: 0.849609375
c2: 0.857421875
tensor(0.4403, device='cuda:0', grad_fn=<NllLossBackward>)
----------------------------------------------------------------------------------------------------
0 500
c1: 0.83

In [29]:
%%time
print(count_acc(bert_model, dev_dataloader))
print(count_acc(bert_model, test_dataloader))

0.8657196184155952
0.8617170923839779
CPU times: user 6min 57s, sys: 2min 46s, total: 9min 44s
Wall time: 9min 44s
